In [1]:
import torch, torchvision
import sys # Python system library needed to load custom functions
import math # module with access to mathematical functions
import os # for changing the directory

import numpy as np  # for performing calculations on numerical arrays
import pandas as pd  # home of the DataFrame construct, _the_ most important object for Data Science
from sklearn.model_selection import train_test_split
import matplotlib.pyplot as plt  # allows creation of insightful plots

sys.path.append('../../audio_preprocessing')
sys.path.append('../../src')
sys.path.append('../../model_training_utils')


from normalised_image_dataset import NormalisedImageDataSet
from gdsc_utils import PROJECT_DIR
import model_training
import model_eval

os.chdir(PROJECT_DIR) # changing our directory to root

In [2]:
df = pd.read_csv('data/normalised_image_train_val_with_upsample.csv')
df.head()

,Unnamed: 0,file_path,label
0,0,data/image_train_val_with_upsample/0.pt,56
1,1,data/image_train_val_with_upsample/1.pt,56
2,2,data/image_train_val_with_upsample/2.pt,56
3,3,data/image_train_val_with_upsample/3.pt,56
4,4,data/image_train_val_with_upsample/4.pt,56


In [3]:
train_df_list = []
val_df_list = []

for i in range(66):
    my_df = df[df["label"] == i]
    current_train_df, current_val_df = train_test_split(my_df, test_size=0.02)
    train_df_list.append(current_train_df)
    val_df_list.append(current_val_df)

df_train = pd.concat(train_df_list, ignore_index=True)
df_val = pd.concat(val_df_list, ignore_index=True)

In [4]:
df_train.shape, df_val.shape

((87548, 3), (1836, 3))

In [5]:
train_dataset = NormalisedImageDataSet(
    df=df_train, 
    shuffle=True
)

val_dataset = NormalisedImageDataSet(
    df=df_val, 
    shuffle=False
)

train_dataloader = torch.utils.data.DataLoader(train_dataset, batch_size=28)
val_dataloader = torch.utils.data.DataLoader(val_dataset, batch_size=28)

In [6]:
device = model_training.get_device()

In [7]:
from torchvision.models import regnet_y_3_2gf
import torch.optim as optim
import torch.nn as nn

#resnet_model = resnet34(weights=ResNet34_Weights.DEFAULT)
regnet_model = regnet_y_3_2gf(num_classes=66)
regnet_model.stem[0] = nn.Conv2d(1, 32, kernel_size=(3, 3), stride=(2, 2), padding=(1, 1), bias=False)
regnet_model = regnet_model.to(device)

In [8]:
print(regnet_model)

RegNet(
  (stem): SimpleStemIN(
    (0): Conv2d(1, 32, kernel_size=(3, 3), stride=(2, 2), padding=(1, 1), bias=False)
    (1): BatchNorm2d(32, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (2): ReLU(inplace=True)
  )
  (trunk_output): Sequential(
    (block1): AnyStage(
      (block1-0): ResBottleneckBlock(
        (proj): Conv2dNormActivation(
          (0): Conv2d(32, 72, kernel_size=(1, 1), stride=(2, 2), bias=False)
          (1): BatchNorm2d(72, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
        )
        (f): BottleneckTransform(
          (a): Conv2dNormActivation(
            (0): Conv2d(32, 72, kernel_size=(1, 1), stride=(1, 1), bias=False)
            (1): BatchNorm2d(72, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
            (2): ReLU(inplace=True)
          )
          (b): Conv2dNormActivation(
            (0): Conv2d(72, 72, kernel_size=(3, 3), stride=(2, 2), padding=(1, 1), groups=3, bias=False)
            

In [9]:
optimizer = optim.Adam(regnet_model.parameters(), amsgrad=True)
loss = nn.CrossEntropyLoss()

In [10]:
model_training.training(
    model=regnet_model, 
    optimizer=optimizer, 
    loss_fn=loss, 
    train_dataloader=train_dataloader, 
    val_dataloader=val_dataloader, 
    model_path="models/regnet_y_3_2gf_train_and_val", 
    start_epoch=14,
    epochs=500,
    early_stop_thresh=50,
)

End of epoch 15: training accuracy = 99.13%, training loss = 0.026710696020228655, training time taken = 520.23 seconds
End of epoch 15: validation accuracy = 98.75%, validation loss = 0.03723608199287341, validation time taken = 10.33 seconds
End of epoch 16: training accuracy = 99.29%, training loss = 0.02166381112735662, training time taken = 527.91 seconds
End of epoch 16: validation accuracy = 98.91%, validation loss = 0.029586624791942417, validation time taken = 5.34 seconds
End of epoch 17: training accuracy = 99.50%, training loss = 0.01628963702162348, training time taken = 527.88 seconds
End of epoch 17: validation accuracy = 99.84%, validation loss = 0.00789103735356052, validation time taken = 10.70 seconds


KeyboardInterrupt: 

In [ ]:
#torch.save(regnet_model, 'models/regnet_y_3_2gf_train_and_val/regnet_model_epoch_11.pth')